In [1]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import itertools
from sklearn.metrics import accuracy_score
print(tf.__version__)
import os
from tensorflow.keras.models import Sequential

2.4.1


In [2]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Dropout, MaxPool2D, Dense, Conv2D, Flatten, BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm
from skimage.transform import resize

In [3]:
from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
from tensorflow.keras import Model

In [4]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3558897114153677129
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3044750132
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14341431770952585801
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [5]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size
def convert_bytes(size, unit=None):
    if unit == "KB":
        return str(round(size / 1024, 3))
    elif unit == "MB":
        return str(round(size / (1024 * 1024), 3)) 
    else:
        return print('File size: ' + str(size) + ' bytes')

In [6]:
device=tf.test.gpu_device_name()
print(device)

/device:GPU:0


In [7]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [8]:
mobilenet_img_size=(224,224,3)

In [9]:
x_test.shape

(10000, 32, 32, 3)

In [10]:
x_train=x_train[:500]
y_train=y_train[:500]
x_test=x_test[:500]
y_test=y_test[:500]

In [11]:
y_train = y_train.flatten()
y_test = y_test.flatten()

In [12]:
y_train

array([6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6, 4, 3,
       6, 6, 2, 6, 3, 5, 4, 0, 0, 9, 1, 3, 4, 0, 3, 7, 3, 3, 5, 2, 2, 7,
       1, 1, 1, 2, 2, 0, 9, 5, 7, 9, 2, 2, 5, 2, 4, 3, 1, 1, 8, 2, 1, 1,
       4, 9, 7, 8, 5, 9, 6, 7, 3, 1, 9, 0, 3, 1, 3, 5, 4, 5, 7, 7, 4, 7,
       9, 4, 2, 3, 8, 0, 1, 6, 1, 1, 4, 1, 8, 3, 9, 6, 6, 1, 8, 5, 2, 9,
       9, 8, 1, 7, 7, 0, 0, 6, 9, 1, 2, 2, 9, 2, 6, 6, 1, 9, 5, 0, 4, 7,
       6, 7, 1, 8, 1, 1, 2, 8, 1, 3, 3, 6, 2, 4, 9, 9, 5, 4, 3, 6, 7, 4,
       6, 8, 5, 5, 4, 3, 1, 8, 4, 7, 6, 0, 9, 5, 1, 3, 8, 2, 7, 5, 3, 4,
       1, 5, 7, 0, 4, 7, 5, 5, 1, 0, 9, 6, 9, 0, 8, 7, 8, 8, 2, 5, 2, 3,
       5, 0, 6, 1, 9, 3, 6, 9, 1, 3, 9, 6, 6, 7, 1, 0, 9, 5, 8, 5, 2, 9,
       0, 8, 8, 0, 6, 9, 1, 1, 6, 3, 7, 6, 6, 0, 6, 6, 1, 7, 1, 5, 8, 3,
       6, 6, 8, 6, 8, 4, 6, 6, 1, 3, 8, 3, 4, 1, 7, 1, 3, 8, 5, 1, 1, 4,
       0, 9, 3, 7, 4, 9, 9, 2, 4, 9, 9, 1, 0, 5, 9, 0, 8, 2, 1, 2, 0, 5,
       6, 3, 2, 7, 8, 8, 6, 0, 7, 9, 4, 5, 6, 4, 2,

In [13]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']



In [14]:
np.isnan(x_train).any()

False

In [15]:
np.isnan(x_train).any()

False

In [16]:
def resize_images(images,img_size):
    resized_images = []
    
    for i in tqdm(range(len(images))):
        resized_images.append(resize(images[i],(img_size),mode = 'constant'))
    
    return np.array(resized_images,dtype='float32')

In [17]:
print('Loading train data for MobileNet :')
x_train = resize_images(x_train,mobilenet_img_size)

print('Loading test data for MobileNet :')
x_test = resize_images(x_test,mobilenet_img_size)

Loading train data for MobileNet :


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:05<00:00, 94.43it/s]


Loading test data for MobileNet :


100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [00:05<00:00, 94.34it/s]


In [18]:
input_shape = (224,224,3)

x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 3)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 3)
x_test=x_test / 255.0

In [19]:
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [20]:
y_train[0]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)>

In [21]:
batch_size = 16
num_classes = 10
epochs = 5

In [22]:
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x
#stem of the model
input = Input(shape = (224,224,3))
x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
x = BatchNormalization()(x)
x = ReLU()(x)
# main part of the model
x = mobilnet_block(x, filters = 64, strides = 1)
x = mobilnet_block(x, filters = 128, strides = 2)
# x = mobilnet_block(x, filters = 128, strides = 1)
# x = mobilnet_block(x, filters = 256, strides = 2)
# x = mobilnet_block(x, filters = 256, strides = 1)
# x = mobilnet_block(x, filters = 512, strides = 2)
# for _ in range (5):
#      x = mobilnet_block(x, filters = 512, strides = 1)
# x = mobilnet_block(x, filters = 1024, strides = 2)
# x = mobilnet_block(x, filters = 1024, strides = 1)
# x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
x=Flatten()(x)
output = Dense (units = 10, activation = 'softmax')(x)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu (ReLU)                 (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 112, 112, 32)      320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 112, 112, 32)      0     

In [23]:
mobilenet_img_size=(224,224,3)

In [24]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32, 3, padding='same', input_shape=x_train.shape[1:], activation='relu'),
#     tf.keras.layers.Conv2D(32, 3, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(64, 3, activation='relu'),
    
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(num_classes, activation='softmax'),
# ])

# model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-06),
#             loss='categorical_crossentropy', metrics=['acc'])

In [25]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(32, 3, padding='same', input_shape=x_train.shape[1:], activation='relu'),
#     tf.keras.layers.Conv2D(32, 3, activation='relu'),
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(64, 3, activation='relu'),
#     tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
#     tf.keras.layers.Conv2D(128, 3, activation='relu'),
    
#     tf.keras.layers.MaxPooling2D(),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(num_classes, activation='softmax'),
# ])

# model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-06),
#             loss='categorical_crossentropy', metrics=['acc'])

In [26]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-06),
            loss='categorical_crossentropy', metrics=['acc'])

In [27]:
x_train.shape

(500, 224, 224, 3)

In [28]:
y_train.shape

TensorShape([500, 10])

In [45]:
with tf.device('/CPU:0'):
    history = model.fit(x_train, y_train, batch_size=16, epochs=50)

Epoch 1/50
32/32 [==============================] - 20s 629ms/step - loss: 1.2693 - acc: 0.7980
Epoch 2/50
32/32 [==============================] - 19s 607ms/step - loss: 1.0478 - acc: 0.8580
Epoch 3/50
32/32 [==============================] - 20s 613ms/step - loss: 0.6189 - acc: 0.8680
Epoch 4/50
32/32 [==============================] - 20s 610ms/step - loss: 0.8317 - acc: 0.8780
Epoch 5/50
32/32 [==============================] - 20s 632ms/step - loss: 0.4163 - acc: 0.9160
Epoch 6/50
32/32 [==============================] - 21s 651ms/step - loss: 0.4633 - acc: 0.9240
Epoch 7/50
32/32 [==============================] - 21s 673ms/step - loss: 0.3677 - acc: 0.9380
Epoch 8/50
32/32 [==============================] - 22s 697ms/step - loss: 0.4680 - acc: 0.9240
Epoch 9/50
32/32 [==============================] - 24s 755ms/step - loss: 0.2559 - acc: 0.9560
Epoch 10/50
32/32 [==============================] - 20s 622ms/step - loss: 0.0550 - acc: 0.9860
Epoch 11/50
32/32 [====================

In [ ]:
# fig, ax = plt.subplots(2,1)
# ax[0].plot(history.history['loss'], color='b', label="Training Loss")
# legend = ax[0].legend(loc='best', shadow=True)

# ax[1].plot(history.history['acc'], color='b', label="Training Accuracy")
# legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
# test_loss, test_acc = model.evaluate(x_test, y_test)

In [ ]:
# # Predict the values from the validation dataset
# y_pred = model.predict(x_test)
# # Convert predictions classes to one hot vectors 
# y_pred_classes = np.argmax(y_pred,axis = 1) 
# # Convert validation observations to one hot vectors
# y_true = np.argmax(y_test,axis = 1)
# # compute the confusion matrix
# confusion_mtx = tf.math.confusion_matrix(y_true, y_pred_classes)

In [ ]:
# plt.figure(figsize=(12, 9))
# c = sns.heatmap(confusion_mtx, annot=True, fmt='g')
# c.set(xticklabels=classes, yticklabels=classes)

In [ ]:
model=MobileNet_model

In [ ]:
model.save("./cifar_mobilenet.h5")

In [ ]:
model.save("./cifar_mobilenet")

In [ ]:
model=tf.keras.models.load_model("./cifar_mobilenet.h5")
test_loss, test_acc = model.evaluate(x_test, y_test)

In [ ]:
model.summary()

In [ ]:
test_acc

In [ ]:
converter=tf.lite.TFLiteConverter.from_saved_model("./cifar_mobilenet")
tflite_model=converter.convert()

In [ ]:
converter=tf.lite.TFLiteConverter.from_saved_model("./cifar_mobilenet")
converter.optimizations=[tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.target_spec.supported_types = [tf.float16]
tflite_quant_model=converter.convert()

In [ ]:
interpreter= tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()




In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], (500, 224,224,3))
interpreter.resize_tensor_input(output_details[0]['index'], (500, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
test_imgs_numpy = np.array(x_test, dtype=np.float32)

In [ ]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

In [ ]:
tflite_model_acc = accuracy_score(prediction_classes, np.argmax(y_test,axis=1))


In [ ]:
interpreter= tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()

In [ ]:
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], (500, 224,224,3))
interpreter.resize_tensor_input(output_details[0]['index'], (500, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

In [ ]:
tflite_model_quant_acc = accuracy_score(prediction_classes, np.argmax(y_test,axis=1))


In [ ]:
keras_model_size = get_file_size("cifar_mobilenet.h5")

In [ ]:
TF_LITE_MODEL_FILE_NAME = "tf_lite_model.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
keras_model_tflite_size=get_file_size("tf_lite_model.tflite")

In [ ]:
TF_LITE_MODEL_FLOAT_16_FILE_NAME = "tf_lite_float_16_model.tflite"
keras_model_tflite_quant_size=open(TF_LITE_MODEL_FLOAT_16_FILE_NAME, "wb").write(tflite_quant_model)

In [ ]:
print("tf model accuracy:",test_acc)
print("tflite model accuracy:",tflite_model_acc)
print("tflite quant model accuracy:",tflite_model_quant_acc)

In [ ]:
print("Keras model size",convert_bytes(keras_model_size,"KB"))
print("Keras tflite model size",convert_bytes(keras_model_tflite_size,"KB"))
print("Keras tflite quant model size",convert_bytes(keras_model_tflite_quant_size,"KB"))


In [ ]:
print("model size",convert_bytes(keras_model_size,"MB"))
print("tflite model size",convert_bytes(keras_model_tflite_size,"MB"))
print("tflite quant model size",convert_bytes(keras_model_tflite_quant_size,"MB"))


In [31]:
import tensorflow_model_optimization as tfmot

In [41]:
MobileNet_model=tfmot.quantization.keras.quantize_model(model)
opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
MobileNet_model.compile(optimizer = opt , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

# Train
MobileNet_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
quantize_layer_4 (QuantizeLa (None, 224, 224, 3)       3         
_________________________________________________________________
quant_conv2d (QuantizeWrappe (None, 112, 112, 32)      961       
_________________________________________________________________
quant_batch_normalization (Q (None, 112, 112, 32)      129       
_________________________________________________________________
quant_re_lu (QuantizeWrapper (None, 112, 112, 32)      3         
_________________________________________________________________
quant_depthwise_conv2d (Quan (None, 112, 112, 32)      323       
_________________________________________________________________
quant_batch_normalization_1  (None, 112, 112, 32)      129   

In [42]:
# model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-06),
#             loss='categorical_crossentropy', metrics=['acc'])
# with tf.device('/gpu:0'):
#     history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)


In [43]:
with tf.device('/CPU:0'):
    mobile = MobileNet_model.fit(x_train, y_train, batch_size = 16, epochs = 50)

Epoch 1/50
32/32 [==============================] - 27s 804ms/step - loss: 2.3007 - accuracy: 0.0868
Epoch 2/50
32/32 [==============================] - 25s 787ms/step - loss: 2.2746 - accuracy: 0.1182
Epoch 3/50
32/32 [==============================] - 25s 788ms/step - loss: 2.2206 - accuracy: 0.1220
Epoch 4/50
32/32 [==============================] - 25s 788ms/step - loss: 2.1538 - accuracy: 0.0722
Epoch 5/50
32/32 [==============================] - 25s 787ms/step - loss: 2.1449 - accuracy: 0.0920
Epoch 6/50
32/32 [==============================] - 25s 789ms/step - loss: 1.9485 - accuracy: 0.1748
Epoch 7/50
32/32 [==============================] - 25s 788ms/step - loss: 1.7855 - accuracy: 0.2029
Epoch 8/50
32/32 [==============================] - 25s 790ms/step - loss: 1.6269 - accuracy: 0.2134
Epoch 9/50
32/32 [==============================] - 25s 790ms/step - loss: 1.7119 - accuracy: 0.2566
Epoch 10/50
32/32 [==============================] - 26s 818ms/step - loss: 1.5831 - accura

In [ ]:
model.save("cifar_aware.h5")

In [ ]:
model.save("./cifar_aware")

In [ ]:
test_loss_aware, test_acc_aware= model.evaluate(x_test, y_test)

In [ ]:
converter=tf.lite.TFLiteConverter.from_saved_model("cifar_aware")
tflite_aware_model=converter.convert()

In [ ]:
interpreter= tf.lite.Interpreter(model_content=tflite_aware_model)
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()




In [ ]:
interpreter.resize_tensor_input(input_details[0]['index'], (10000, 32, 32,3))
interpreter.resize_tensor_input(output_details[0]['index'], (10000, 10))
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
interpreter.set_tensor(input_details[0]['index'], test_imgs_numpy)
interpreter.invoke()
tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)
prediction_classes = np.argmax(tflite_model_predictions, axis=1)

In [ ]:
tflite_aware_model_acc = accuracy_score(prediction_classes, np.argmax(y_test,axis=1))


In [ ]:
tflite_aware_model_acc